# Rt2_assignment2

Inside this notebook it is implemented an updated version of the user_interface which previously controlled the behavior of the robot. Each block is dedicated to a specific task and there are all described specifically. 

## How to run the code

After cloning the repository from [github](https://github.com/lauratriglia/rt2_assignment1), checkout on the branch "doxy". Once you're in the branch, two terminals are required. In the first one you have to run the simulation, writing the following command:
``` 
roslaunch rt2_assignment1 sim.launch
```
In the second one, you need to start jupyter, writing the following command:
```
cd rt2_assignment1/notebook

~rt2_assignment1/notebook/ jupyter notebook --allow-root
```
Once the notebook is open, run each block in order.

## A brief explanation of the assignment

The aim of this assignment was to update the version of the previous user_interface.py, adding buttons and sliders to control the behavior of the robot in term of modifying linear and angular velocities and its odometry. You will find in the _Control Panel_ sliders to control the linear and angular velocities, and also the start/stop buttons to see the robot moving. More description can be found in the dedicated cell. 
Beside controlling the behaviour of the robot, 4 different graphs will be shown:
* **Position and orientation**: it shows the robot position and orientation
* **Velocities**: it shows linear and angular velocities comparing the one that you can set with the slider and the real one that the robot has. 
* **Reached vs Cancelled**: it shows how many goals are reached and how many are cancelled
* **Time to Reach Goal**: it shows how much time is needed to reached the goal

# Code

### Libraries

In the following block all the necessary libraries are declared. 

In [1]:
import rospy
import time
import jupyros as jr
import matplotlib
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import numpy as np 
import math
import actionlib
import actionlib.msg
import ipywidgets as widgets
from rt2_assignment1.srv import Command, SetVel
from geometry_msgs.msg import Twist
from nav_msgs.msg import Odometry


%matplotlib widget
from matplotlib.animation import FuncAnimation
from matplotlib import animation, rc
from std_msgs.msg import String, Float64, Bool, Float32
from IPython.display import display
from ipywidgets import Button, GridBox, Layout, ButtonStyle, FloatSlider, HBox, VBox
from tf import transformations


## Classes

### ValuePlots

This class generates queues that has a fixed maximum size. THe scope of this class is to store all the most recent data necessary for the graphs.

In [2]:
class ValuePlots(list):
    def __init__(self, max_size, l=[]):
        super().__init__(l)
        self.max_size = max_size
    def append(self, data):
        super().append(data)
        if super().__len__() > self.max_size:
            super().pop(0)

### Direction Pad

This class is used to control the four directional buttons which are dedicated to control the behaviour of the robot 

In [3]:
class DirectionPad:
    def __init__(self, lin_slider, ang_slider):
        self.vel_pub = rospy.Publisher('/cmd_vel', Twist, queue_size=1)
        self.twist = Twist()
        self.go_on_mov = 0.0
        self.ang_mov = 0.0
        self.lin_slider = lin_slider
        self.ang_slider = ang_slider
    
    def goForward(self):
        self.go_on_mov = 1.0
        self.ang_mov = 0.0
    def goBackward(self):
        self.go_on_mov = -1.0
        self.ang_mov = 0.0
    def turnRight(self):
        self.go_on_mov = 0.0
        self.ang_mov = 1.0
    def turnLeft(self):
        self.go_on_mov = 0.0
        self.ang_mov = -1.0
    def stopping(self):
        self.go_on_mov = 0.0
        self.ang_mov = 0.0
    
    def pubTwist(self):
        self.twist.linear.x = self.go_on_mov*lin_slider.value
        self.twist.linear.z = self.ang_mov*ang_slider.value
        
        self.vel_pub.publish(self.twist)

## Global variables

In [4]:
output = widgets.Output()

lin_init = 0.6
ang_init = 0.6

maxLen_odm = 2000
odm_x = ValuePlots(maxLen_odm)
odm_y = ValuePlots(maxLen_odm)
odm_rot = 0

velMax = 200
vel_x_sim = ValuePlots(velMax, [lin_init]*velMax)
vel_x_odom = ValuePlots(velMax, [0]*velMax)
vel_w_sim = ValuePlots(velMax, [ang_init]*velMax)
vel_w_odom = ValuePlots(velMax, [0]*velMax)

t = np.linspace(0,1,velMax)

from enum import Enum

class RobotB(Enum):
    STOP = 0
    GO = 1
    PAD = 2
behavior = RobotB.STOP

vel_client = None
ui_client = None

## Widgets

### Sliders

Sliders are used to set the maximum velocities.

In [5]:
max_vel = 1.6
lin_slider = FloatSlider(description='Linear Velocity', value=lin_init, min=0.1, max=max_vel, step=0.02)
ang_slider = FloatSlider(description='Angular Velocity', value=ang_init, min=0.1, max=max_vel, step=0.02)

v1=VBox([lin_slider, ang_slider])
def lin_vel_change(vel):
    vel_client(vel["new"], vel_w_sim[-1])

def ang_vel_change(vel):
    vel_client(vel_x_sim[-1], vel["new"])
    
lin_slider.observe(lin_vel_change, names='value')
ang_slider.observe(ang_vel_change, names='value')

### Buttons

In this cell all the buttons are initiliazed, both the arrow ones and the start/stop. 

In [12]:
up_Button = Button(description='\u2191', 
                  layout=Layout(width='auto', grid_area='up'),
                  style=ButtonStyle(button_color='lightblue'))
down_Button = Button(description='\u2193',
                  layout=Layout(width='auto', grid_area='down'),
                  style=ButtonStyle(button_color='lightblue'))
right_Button = Button(description='\u2192', 
                  layout=Layout(width='auto', grid_area='right'),
                  style=ButtonStyle(button_color='lightblue'))
left_Button = Button(description='\u2190', 
                  layout=Layout(width='auto', grid_area='left'),
                  style=ButtonStyle(button_color='lightblue'))


gridButtons = GridBox(children=[up_Button, down_Button, right_Button, left_Button],
        layout=Layout(
            width='100%',
            grid_template_rows='auto auto',
            grid_template_columns='33% 33% 33%',
            grid_template_areas='''
            ". up ."
            "left down right"
            ''')
        )

start_Button = Button(description='START', layout=Layout(width='100%',grid_area='start'),
                  style=ButtonStyle(button_color='lightblue'))
stop_Button = Button(description='STOP', layout=Layout(width='100%',grid_area='stop'),
                  style=ButtonStyle(button_color='lightblue'))
h1=HBox([start_Button,stop_Button])

gridAll = GridBox(children=[v1, h1, gridButtons],
            layout=Layout(
                width='100%',
                grid_template_row='auto auto',
                grid_template_columns='33% 40% 27%',
                grid_templates_areas='''
                "v1 . gridButtons"
                " . h1 ."
                ''')
            )

## Functions

In [13]:
dir_pad = DirectionPad(lin_slider, ang_slider)

def start_gotopoint(b):
    global behavior
    if behavior != RobotB.GO:
        ui_client("start")
        behavior = RobotB.GO

def stop_motion(b):
    global behavior
    if behavior == RobotB.GO:
        ui_client("stop")
    elif behavior == RobotB.PAD:
        dir_pad.stop()
        dir_pad.pubTwist()
        
    behavior = RobotB.STOP

def  directionalButton(b):
    global behavior
    if behavior == RobotB.GO:
        stop_motion(b)
    if b.layout.grid_area == 'up':
        dir_pad.goForward()
    elif b.layout.grid_area == 'down':
        dir_pad.goBackward()
    elif b.layout.grid_area == 'right':
        dir_pad.turnRight()
    elif b.layout.grid_area == 'left':
        dir_pad.turnLeft()
    else:
        return 

    behavior = RobotB.PAD

def odomFunction(msg):
    global odm_rot, odm_x, odm_y
    
    global behavior
    odm_x.append(msg.pose.pose.position.x)
    odm_y.append(msg.pose.pose.position.y)
    
    quaternion = (
        msg.pose.pose.orientation.x,
        msg.pose.pose.orientation.y,
        msg.pose.pose.orientation.z,
        msg.pose.pose.orientation.w)
    euler = transformations.euler_from_quaternion(quaternion)
    odm_rot = euler[2]
    
    vel_x_sim.append(lin_slider.value)
    vel_x_odom.append(msg.twist.twist.linear.x)
    
    vel_w_sim.append(ang_slider.value)
    vel_w_odom.append(msg.twist.twist.angular.x)
    
    if behavior == RobotB.PAD:
        dir_pad.pubTwist()

### Buttons binding

In [14]:
start_Button.on_click(start_gotopoint)
stop_Button.on_click(stop_motion)
up_Button.on_click(directionalButton)
down_Button.on_click(directionalButton)
left_Button.on_click(directionalButton)
right_Button.on_click(directionalButton)

## Ros node start

In [15]:
rospy.init_node("jupyter_interface")

sub = rospy.Subscriber('/odom', Odometry, odomFunction)

rospy.wait_for_service('/user_interface')
ui_client = rospy.ServiceProxy('/user_interface', Command)
rospy.wait_for_service('/set_vel')
vel_client = rospy.ServiceProxy('/set_vel', SetVel)

## Control panel

In [16]:
display(gridAll)
output

GridBox(children=(VBox(children=(FloatSlider(value=0.6, description='Linear Velocity', max=1.6, min=0.1, step=…

Output()

In [17]:
from rt2_assignment1.msg import PoseActionResult
number_goal={'succ':0,'canc':0}
time_goal=0

def goalCallBack(msg):
    global number_goal
    if msg.result.effect:
        number_goal['succ']+=1
        duration = msg.header.stamp - msg.status.goal_id.stamp
        time_goal.append(duration.to_sec())
    else:
        number_goal['canc']+=1
    
goal_sub= rospy.Subscriber('/go_to_point/result', PoseActionResult, goalCallBack)
    

# Plots

## Position and orientation

In [23]:
fig, ax = plt.subplots()

ax.set_xlim((-6, 6))
ax.set_ylim((-6, 6))

odom_plt, = ax.plot([], [], 'g-')
arrw_plt, = ax.plot([], [], 'b-.', linewidth=2.5)
arrw_len_ = 0.7

ax.set_title("Position and orientation of the robot")
ax.set_xlabel("X")
ax.set_ylabel("Y")

def init():
    odom_plt.set_data([], [])
    arrw_plt.set_data([], [])
    return (odom_plt, arrw_plt)

def animate(i):
    odom_plt.set_data(odm_x, odm_y)
    n_x = odm_x[-1] + math.cos(odm_rot)*arrw_len_
    n_y = odm_y[-1] + math.sin(odm_rot)*arrw_len_
    
    arrw_plt.set_data((odm_x[-1], n_x), (odm_y[-1], n_y))
    
    return (odom_plt, arrw_plt)

anim1 = animation.FuncAnimation(fig, animate, init_func = init, frames=50, interval=20, blit=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Velocities

In [24]:
fig_vel, axs = plt.subplots(nrows=2)

ax0, ax1 = axs.flatten()
ax0.set_title("Linear velocity")
ax0.set_ylim(0.0, max_vel)
ax0.set_xticks([])

vel_x_plt_sim, = ax0.plot([], [], 'r', label='max linear velocity')
vel_x_plt_odm, = ax0.plot([], [], 'g', label='current linear velocity', linewidth=2.0)

ax1.set_title("Angular velocity")
ax1.set_ylim(0.0, max_vel)
ax1.set_xticks([])

vel_w_plt_sim, = ax1.plot([], [], 'r', label='max angular velocity')
vel_w_plt_odm, = ax1.plot([], [], 'g', label='current angular velocity', linewidth=2.0)


def init_vel():
    vel_x_plt_sim.set_data([], [])
    vel_x_plt_odm.set_data([], [])
    
    vel_w_plt_sim.set_data([], [])
    vel_w_plt_odm.set_data([], [])
    return (vel_x_plt_sim, vel_x_plt_odm, vel_w_plt_sim, vel_w_plt_odm)

def animate_vel(i):
    vel_x_plt_sim.set_data(t, vel_x_sim)
    vel_x_plt_odm.set_data(t, vel_x_odom)
    
    vel_w_plt_sim.set_data(t, vel_w_sim)
    vel_w_plt_odm.set_data(t, vel_w_odom)
    
    return (vel_x_plt_sim, vel_x_plt_odm, vel_w_plt_sim, vel_w_plt_odm)

anim_vel = animation.FuncAnimation(fig_vel, animate_vel, init_func = init_vel, frames=100, interval=20, blit=True)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Reached vs Cancelled

In [25]:
fig_bar, ax_bar = plt.subplots()

ax_bar.set_title("Reached vs Cancelled goals")
ax_bar.set_ylim(0, 50)

ind = np.array(['Reached','Cancelled'])

bar_plt = ax_bar.bar(ind, [0,0], width=0.3)

def init_bar():
    for bb, ii, col in zip(bar_plt, number_goal.values(), ('g', 'b')):
        bb.set_height(ii)
        bb.set_color(col)
    return (bar_plt,)

def animate_bar(i):
    for bb, ii in zip(bar_plt, number_goal.values()):
        bb.set_height(ii)
    
    return (bar_plt,)

anim_bar = animation.FuncAnimation(fig_bar, animate_bar, init_func = init_bar, frames=100, interval=20, blit=True)




Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Time to reach

In [21]:
fig_time, ax_time = plt.subplots()

ax_time.set_title("Time to reach the goal")

ax_time.set_xlabel("time")
ax_time.set_ylabel("goals")
ax_time.set_ylim((0, 5))
ax_time.set_xlim((0, 40))

n_bins = 10;
time_plt, _, _ = ax_time.hist([], bins=n_bins, align='mid')

def animate_time(i):
    ax_time.cla()
    ax_time.set_title("Time to reach the goal")

    ax_time.set_xlabel("time")
    ax_time.set_ylabel("goals")
    ax_time.set_ylim((0, 5))
    ax_time.set_xlim((0, 40))
    time_plt, _, _ = ax_time.hist(goal_time, bins=n_bins, align='mid')
    return (time_plt,)

anim_time = animation.FuncAnimation(fig_time, animate_time, frames=100, interval=200, blit=True)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …